In [1]:
###########################
# IMPORT
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV, cross_val_score
######################################

#Caricamento dati

In [6]:
#train_df = pd.read_csv("train_net.csv")
train_df = pd.read_csv("trafficoGenerato.csv")

#Elaborazione dati 

In [ ]:
from sklearn.model_selection import train_test_split

#columns_to_exclude = ['SERVICE','IPV4_SRC_ADDR', 'IPV4_DST_ADDR','ANALYSIS_TIMESTAMP']
columns_to_exclude = ['ip_src', 'ip_dst']
train_data = train_df.drop(columns=columns_to_exclude)

# Prepara le feature e il target per il training set
X_train = train_data.drop('alert', axis=1)
y_train = train_data['alert']


# Gestisci i valori NaN nelle feature
X_train = X_train.fillna(X_train.mean(numeric_only=True)).fillna('MISSING')

# Gestisci i valori NaN nel target
y_train = y_train.fillna(y_train.mode()[0])

# Create dummy columns for the PROTOCOL MAP column

flags_dummies = pd.get_dummies(X_train['flags'], prefix='flags', drop_first=True) # drop_first=True per evitare la multi-collinearità

# Rimuovi le colonne originali e aggiungi le nuove colonne numeriche
X_train = X_train.drop(['flags'], axis=1)  # Rimuove le colonne testuali
X_train = pd.concat([X_train, flags_dummies], axis=1)

#print(X_train.dtypes)

def split_maintain_distribution(X, y):
  sss=StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=9)
  indexes = sss.split(X, y)
  train_indices, test_indices = next(indexes)
  return X.iloc[train_indices], X.iloc[test_indices], y.iloc[train_indices], y.iloc[test_indices]

X_train, X_val, y_train, y_val = split_maintain_distribution(X_train, y_train)

print('Train set distribution:')
print(y_train.value_counts(normalize=True))
print()
print('Validation set distribution:')
print(y_val.value_counts(normalize=True))


Train set distribution:
alert
No Alert     0.780945
Port Scan    0.219055
Name: proportion, dtype: float64

Validation set distribution:
alert
No Alert     0.782468
Port Scan    0.217532
Name: proportion, dtype: float64


#Data scaling

In [13]:
from sklearn.preprocessing import StandardScaler
import joblib

# Fix scaler on train set
scaler = StandardScaler()
fitter = scaler.fit(X_train)

joblib.dump(fitter, 'scaler.joblib')

# Scale train and validation sets
x_train_scaled = fitter.transform(X_train)
x_validation_scaled = fitter.transform(X_val)

# Convert to pandas dataframe
df_feat_train = pd.DataFrame(x_train_scaled, columns=X_train.columns)
df_feat_validation = pd.DataFrame(x_validation_scaled, columns=X_val.columns)

#Addestramento con Random Forest

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from joblib import dump

# Inizializzazione del classificatore Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Addestramento del modello
rf_model.fit(df_feat_train, y_train)

# Salva il modello addestrato in un file .joblib
dump(rf_model, 'modello_addestrato_datiGenerati.joblib')

# Valutazione sul validation set
val_predictions = rf_model.predict(df_feat_validation)
val_accuracy = accuracy_score(y_val, val_predictions) * 100
print(f'Validation Accuracy: {val_accuracy:.2f}%')


Validation Accuracy: 100.00%


#Valutazione modello

In [14]:
from sklearn.metrics import classification_report, confusion_matrix

# Calcolo di precision, recall e F1-score
print("Classification Report:")
print(classification_report(y_val, val_predictions))

# Calcolo della matrice di confusione
conf_matrix = confusion_matrix(y_val, val_predictions)
print("Confusion Matrix:")
print(conf_matrix)

Classification Report:
              precision    recall  f1-score   support

         DOS       1.00      1.00      1.00       134
    No Alert       1.00      1.00      1.00       198

    accuracy                           1.00       332
   macro avg       1.00      1.00      1.00       332
weighted avg       1.00      1.00      1.00       332

Confusion Matrix:
[[134   0]
 [  0 198]]
